# Modelagem Parte 1: Árvore de Decisão e KNN 

**Responsável:** Clara

**Objetivo:** Nesta etapa, realizamos o treinamento dos algoritmos **Decision Tree** e **K-Nearest Neighbors (KNN)** utilizando o dataset pré-processado de saúde fetal.

**Metodologia:**
* **Validação:** 10-fold Stratified Cross-Validation.
* **Otimização:** `GridSearch` para encontrar os melhores hiperparâmetros (testando no mínimo 3 combinações).
* **Métricas:** Acurácia, Precisão, Recall e F1-Score (*weighted*).

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import sys
import os

# Adiciona a raiz do projeto ao PYTHONPATH
sys.path.append(os.path.abspath(".."))


from src.modeling import run_classification_experiment

data_path = os.path.join('..', 'data', 'processed', 'fetal_health_processed.csv')
df = pd.read_csv(data_path)

X = df.drop('fetal_health', axis=1)
y = df['fetal_health']

print("Dados carregados com sucesso!")

## 1. Árvore de Decisão (Decision Tree)

A árvore de decisão é um modelo interpretável que divide os dados com base em regras de decisão. Para evitar *overfitting* e encontrar a melhor generalização, testaremos os seguintes hiperparâmetros via `GridSearch`:

* **Criterion:** `gini` vs `entropy` (avalia qual métrica de pureza funciona melhor na divisão dos nós).
* **Max Depth:** `None` (sem limite), `10`, `20`, `30` (controla a profundidade máxima para limitar a complexidade da árvore).
* **Min Samples Split:** Controla o número mínimo de amostras necessárias para dividir um nó interno.

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)

dt_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20]
}

# Chama a função que está no arquivo .py
dt_grid = run_classification_experiment(dt_model, dt_params, X, y, 'DecisionTree')

print(f"Melhores parâmetros DT: {dt_grid.best_params_}")

## 2. K-Nearest Neighbors (KNN)

O KNN classifica instâncias com base na proximidade com exemplos vizinhos. É sensível à escala dos dados (etapa já tratada no pré-processamento). Testaremos:

* **N Neighbors (k):** `3`, `5`, `7`, `9`, `11`, `15` (número de vizinhos considerados).
* **Weights:** `uniform` (todos têm peso igual) vs `distance` (vizinhos mais próximos têm maior influência na votação).
* **Metric:** `euclidean` (distância padrão) vs `manhattan` (soma das diferenças absolutas).

In [ ]:
knn_model = KNeighborsClassifier()

knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn_grid = run_classification_experiment(knn_model, knn_params, X, y, 'KNN')

print(f"Melhores parâmetros KNN: {knn_grid.best_params_}")